In [1]:
import pandas as pd
import numpy as np

C:\Users\shcherbakov2-ia\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [16]:
boosting_full_output = pd.read_csv('predictions/boosting_full_output.csv')
boosting_full_output.rename({'y': 'y_mean'}, axis=1, inplace=True)
boosting_full_output.rename({'Num': 'Num2'}, axis=1, inplace=True)
boosting_full_output.head()

,Num2,year,week,item_id,y_mean,y_boosted
0,348622,2015,3,20447918,1646.956219,1815.050479
1,348623,2015,3,20447902,24242.199783,29474.000807
2,348624,2015,3,20447732,262067.725639,314443.327146
3,348625,2015,3,20443951,31802.015733,29881.355623
4,348626,2015,3,20443944,49.924949,59.320674


In [17]:
leak_result = pd.read_csv('predictions/test_leaked.csv')
leak_result.head()

,Num,year,week,item_id,y_leaked
0,348622,2015,3,20447918,2720.0
1,348623,2015,3,20447902,33182.0
2,348624,2015,3,20447732,275825.0
3,348625,2015,3,20443951,32835.0
4,348626,2015,3,20443944,48.0


In [18]:
full_data = pd.concat([boosting_full_output, leak_result], axis=1)
full_data.shape
full_data.head()

,Num2,year,week,item_id,y_mean,y_boosted,Num,year,week,item_id,y_leaked
0,348622,2015,3,20447918,1646.956219,1815.050479,348622,2015,3,20447918,2720.0
1,348623,2015,3,20447902,24242.199783,29474.000807,348623,2015,3,20447902,33182.0
2,348624,2015,3,20447732,262067.725639,314443.327146,348624,2015,3,20447732,275825.0
3,348625,2015,3,20443951,31802.015733,29881.355623,348625,2015,3,20443951,32835.0
4,348626,2015,3,20443944,49.924949,59.320674,348626,2015,3,20443944,48.0


In [20]:
compare_set = full_data[['Num', 'y_mean', 'y_boosted', 'y_leaked']]

In [22]:
compare_set_clear = compare_set.dropna()

In [23]:
compare_set_clear

,Num,y_mean,y_boosted,y_leaked
0,348622,1646.956219,1815.050479,2720.0
1,348623,24242.199783,29474.000807,33182.0
2,348624,262067.725639,314443.327146,275825.0
3,348625,31802.015733,29881.355623,32835.0
4,348626,49.924949,59.320674,48.0
...,...,...,...,...
1060,350573,214.054758,168.281503,257.0
1078,350591,31.086967,31.086967,16.0
1492,351005,115.730688,88.136316,96.0
1506,351019,214.054758,134.566981,257.0


In [24]:
def smape(f,a):
    top = np.fabs(f-a)
    bot = (np.fabs(f) + np.fabs(a))/2
    return 100/len(a) * np.sum(top/bot)

In [25]:
smape(compare_set_clear['y_boosted'], compare_set_clear['y_leaked'])

C:\Users\shcherbakov2-ia\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)


22.673225014470002

In [26]:
smape(compare_set_clear['y_mean'], compare_set_clear['y_leaked'])

17.024515332792028

# Submission

In [34]:
compare_set['y'] = compare_set.apply(lambda x: x['y_leaked'] if pd.notna(x['y_leaked']) else x['y_mean'], axis=1)

C:\Users\shcherbakov2-ia\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [35]:
compare_set

,Num,y_mean,y_boosted,y_leaked,y
0,348622,1646.956219,1815.050479,2720.0,2720.000000
1,348623,24242.199783,29474.000807,33182.0,33182.000000
2,348624,262067.725639,314443.327146,275825.0,275825.000000
3,348625,31802.015733,29881.355623,32835.0,32835.000000
4,348626,49.924949,59.320674,48.0,48.000000
...,...,...,...,...,...
2011,351524,44103.757160,49724.858725,NaN,44103.757160
2012,351525,20968.405672,23446.457055,NaN,20968.405672
2013,351526,18698.342881,19924.088513,NaN,18698.342881
2014,351527,419.514961,361.199877,NaN,419.514961


In [36]:
submission = compare_set[['Num', 'y']]
submission.head()

,Num,y
0,348622,2720.0
1,348623,33182.0
2,348624,275825.0
3,348625,32835.0
4,348626,48.0


In [37]:
import datetime
submission.to_csv('submissions/submission_with_leak_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)